In [1]:
import numpy as np
from helper_functions import recall, recall12, mean_recall, calculate_recall_per_customer_batch, calculate_recall_per_week
from helper_functions import read_parquet_datasets
from candidate_generation import get_candidates
import pandas as pd
import time

transactions, customers, articles = read_parquet_datasets()

transactions_copy = transactions.copy()
validation = transactions[transactions.week == transactions.week.max()]

test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [2]:
c2weeks = transactions.groupby('customer_id')['week'].unique()
c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week
    
candidates_last_purchase = transactions.copy()
weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
candidates_last_purchase.week=weeks

### Create three features: age, index_group_name and mean purchase price of the customer.

In [3]:
# first create mean price per customer
mean_price_per_c = transactions.groupby('customer_id')['price'].mean().rename('mean_price_per_c').reset_index()
# figure out the customers preferred index group name by looking at past transactions articles' index_group_name
temp_transactions = pd.merge(transactions, articles[['article_id', 'index_group_name']], on='article_id', how='left')
# per customer count the number of purchases of each index_group_name
c2index_group_name = temp_transactions.groupby('customer_id')['index_group_name'].value_counts().rename('count').reset_index()
c2index_group_name = c2index_group_name.sort_values('count', ascending=False).groupby('customer_id').head(1).rename(columns={'index_group_name': 'highest_count_ign_per_c'})
# merge the mean_price_per_c and c2index_group_name dataframes
transactions_with_2feat = pd.merge(c2index_group_name[['customer_id', 'highest_count_ign_per_c']], mean_price_per_c, on='customer_id', how='left')

# per article, find the average age of the customers who bought it
transactions_with_age = pd.merge(transactions, customers[['customer_id', 'age']], on='customer_id', how='left')
mean_age_per_article = transactions_with_age.groupby('article_id')['age'].mean().rename('mean_age_per_a').astype('int8').reset_index()

# merge to get the mean_price_per_c, highest_count_ign_per_c and mean_age_per_a columns. This forms the transactions_with_3feat (transactions with three features) dataframe
transactions_with_age_2feat = pd.merge(transactions_with_age, transactions_with_2feat, on='customer_id', how='left')
transactions_with_3feat = pd.merge(transactions_with_age_2feat, mean_age_per_article, on='article_id', how='left')
transactions_with_3feat = pd.merge(transactions_with_3feat, articles[['article_id', 'index_group_name']], on='article_id', how='left') # to get the index_group_name (back)

In [4]:
transactions_with_3feat

,t_dat,customer_id,article_id,price,sales_channel_id,week,age,highest_count_ign_per_c,mean_price_per_c,mean_age_per_a,index_group_name
0,2020-07-15,272412481300040,778064028,0.008458,1,95,48,0,0.030597,32,0
1,2020-07-15,272412481300040,816592008,0.016932,1,95,48,0,0.030597,37,2
2,2020-07-15,272412481300040,621381021,0.033881,1,95,48,0,0.030597,35,2
3,2020-07-15,272412481300040,817477003,0.025407,1,95,48,0,0.030597,33,0
4,2020-07-15,272412481300040,899088002,0.025407,1,95,48,0,0.030597,35,2
...,...,...,...,...,...,...,...,...,...,...,...
2762867,2020-09-22,18439937050817258297,891591003,0.084729,2,104,49,0,0.040379,41,0
2762868,2020-09-22,18439937050817258297,869706005,0.084729,2,104,49,0,0.040379,39,0
2762869,2020-09-22,18440902715633436014,918894002,0.016932,1,104,18,0,0.028232,30,0
2762870,2020-09-22,18440902715633436014,761269001,0.016932,1,104,18,0,0.028232,33,0


In [5]:
# merge t with customers columns customer_id, age on customer_id for later use
t_merged = pd.merge(transactions_with_2feat, customers[['customer_id', 'age']], on='customer_id', how='left')
# Create a DataFrame with unique articles
unique_articles = transactions_with_3feat[['t_dat', 'sales_channel_id', 'article_id', 'price', 'index_group_name', 'week']].drop_duplicates()
# we only keep latest purchase of each article
unique_articles = unique_articles.sort_values('t_dat', ascending=False).drop_duplicates('article_id')

### Calculate overall mean recall

In [6]:
# Define the total number of customers and batch size
total_customers = mean_price_per_c['customer_id'].nunique()
print("Total customers:", total_customers)
batch_size = 1000

validation = transactions # we use all (actually the last 5 weeks) as validation

topX_price=50 # top x candidates filtered by price columns
topX_age=25 # top x candidates filtered by age columns

# Initialize an empty list to store mean recalls for each batch
mean_recalls = {}

batches = mean_price_per_c['customer_id'].unique()

# Iterate over customer batches
for i in range(0, total_customers, batch_size):
    start = time.time()
    start_idx = i
    end_idx = i + batch_size
    
    # Get the current batch of customers
    current_batch = batches[start_idx:end_idx]
    
    # Get candidates for the current batch
    top_candidates_3feat = get_candidates([current_batch], unique_articles, t_merged, mean_age_per_article, topX_price, topX_age)
    
    # Increment the week for the candidates to match the next week
    top_candidates_3feat_prev_week = top_candidates_3feat.copy()
    top_candidates_3feat_prev_week.week += 1
    
    # Calculate mean recall for the current batch
    mean_recall_per_week = calculate_recall_per_week(validation, top_candidates_3feat_prev_week, current_batch, topX_age, compare_against_bestsellers=True)
    
    # Append mean recall to the dictionary
    for week, mean_recall in mean_recall_per_week.items():
        if week not in mean_recalls:
            mean_recalls[week] = []
        mean_recalls[week].append(mean_recall)
    
    print(f'Batch {i} processed in {time.time() - start:.2f} seconds')

# Calculate overall mean recall
mean_recalls = {week: np.mean(mean_recall) for week, mean_recall in mean_recalls.items()}

Total customers: 437365
Chunk 0 processed in 20.32 seconds
Batch 0 processed in 20.42 seconds
Chunk 0 processed in 19.80 seconds
Batch 1000 processed in 19.90 seconds
Chunk 0 processed in 25.16 seconds
Batch 2000 processed in 25.36 seconds
Chunk 0 processed in 21.71 seconds
Batch 3000 processed in 21.81 seconds
Chunk 0 processed in 19.85 seconds
Batch 4000 processed in 19.95 seconds
Chunk 0 processed in 18.65 seconds
Batch 5000 processed in 18.75 seconds
Chunk 0 processed in 19.68 seconds
Batch 6000 processed in 19.78 seconds
Chunk 0 processed in 19.65 seconds
Batch 7000 processed in 19.75 seconds
Chunk 0 processed in 19.21 seconds
Batch 8000 processed in 19.32 seconds
Chunk 0 processed in 19.34 seconds
Batch 9000 processed in 19.44 seconds
Chunk 0 processed in 19.52 seconds
Batch 10000 processed in 19.62 seconds
Chunk 0 processed in 19.55 seconds
Batch 11000 processed in 19.67 seconds
Chunk 0 processed in 19.36 seconds
Batch 12000 processed in 19.47 seconds
Chunk 0 processed in 19.66 

In [7]:
print(mean_recalls)

{104: 0.0, 103: 0.0, 102: 0.0, 101: 0.0, 100: 0.0}
